In this project I'm goint to use the S&P 500 index,  a stock market index that measures the stock performance of 500 large companies listed on stock exchanges in the United States (see [here in wikipedia](https://en.wikipedia.org/wiki/S%26P_500_Index)). <br>

I'll train a lineare regression model with data until 1/1/2013 and I'll use the rest of the data to test if the prediction model is effective. <br>
This is just an experiment, predicting stock prices using machine learning models is a very difficult task, beyond the complexity of the models used in this project.

In [50]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


In [51]:
stocks = pd.read_csv("sphist.csv", parse_dates=["Date"])

In [52]:
stocks.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [53]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16590 non-null  datetime64[ns]
 1   Open       16590 non-null  float64       
 2   High       16590 non-null  float64       
 3   Low        16590 non-null  float64       
 4   Close      16590 non-null  float64       
 5   Volume     16590 non-null  float64       
 6   Adj Close  16590 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 907.4 KB


In [54]:
stocks = stocks.sort_values(by=["Date"])

### Generating indicators

I'll create new features from the ones in the dataset to help in predictions. Features indexing previous days are calculated not including the day they are referred to, so the day_5 column is the average of the five working days preceding the current one.

### Index at Close

#### Average index at close in the previous 5 working days

In [55]:
stocks["days_5"] = stocks["Close"].rolling(window=5).mean().shift(1)

#### Standard deviation of the average of the index at close in the previous 5 days

In [56]:
stocks["std_5"] = stocks["Close"].rolling(window=5).std().shift(1)

#### Average index at close in the previous 30 working days

In [57]:
stocks["days_30"] = stocks["Close"].rolling(window=30).mean().shift(1)

#### Standard deviation of the average of the index at close in the previous 30 days

In [58]:
stocks["std_30"] = stocks["Close"].rolling(window=30).std().shift(1)

#### Average index at close in the previous 365 working days

In [59]:
stocks["days_365"] = stocks["Close"].rolling(window=365).mean().shift(1)

#### Standard deviation of the average of the index at close in the previous 365 days

In [60]:
stocks["std_365"] = stocks["Close"].rolling(window=365).std().shift(1)

#### Ratio between the the five days average and the 365 days average of the index at close

In [61]:
stocks["ratio_5_365"] = stocks["days_5"] / stocks["days_365"]

### Ratio between std_5 and std_365

In [62]:
stocks["std_ratio_5_365"] = stocks["std_5"]/stocks["std_365"]

### Volume

#### Volume in the previous 5 working days

In [63]:
stocks["volume_5"] = stocks["Volume"].rolling(window=5).mean().shift(1)

#### Standard deviation of the average volume over the last 5 days

In [64]:
stocks["std_vol_5"] = stocks["volume_5"].rolling(window=5).std().shift(1)

#### Volume in the previous 365  working days

In [65]:
stocks["volume_365"] = stocks["Volume"].rolling(window=365).mean().shift(1)

#### Standard deviation of the average volume over the last 365 days

In [66]:
stocks["std_vol_365"] = stocks["volume_365"].rolling(window=365).std().shift(1)

#### Ratio between volume in the previous 5 and 365 working days

In [67]:
stocks["vol_ratio_5_365"] = stocks["volume_5"] /stocks["volume_365"]

#### Ratio between std_vol_5 and std_vol_365

In [68]:
stocks["std_vol_ratio_5_365"] = stocks["std_vol_5"]/stocks["std_vol_365"]

### Yearly indexes

#### Yearly min and max indexes

In [69]:
stocks["min_365"] = stocks["Close"].rolling(window=365).min().shift(1)

In [70]:
stocks["max_365"] = stocks["Close"].rolling(window=365).max().shift(1)

In [71]:
stocks["ratio_to_min_365"] = stocks["Close"] /stocks["min_365"]

In [72]:
stocks["ratio_to_max_365"] = stocks["Close"] /stocks["max_365"]

In [73]:
stocks.shape

(16590, 25)

In [74]:
data = stocks.copy()

### Removing null values and Scaling the data

I'll remove any rows before 1951-01-03 and any rows with missing values

In [75]:
data = data[data["Date"] > datetime(year=1951, month=1, day=2)]

In [76]:
data.shape

(16340, 25)

In [77]:
data = data.dropna(axis= 0)

In [78]:
data.shape

(15860, 25)

In [79]:
data.iloc[:, 0:25]

,Date,Open,High,Low,Close,Volume,Adj Close,days_5,std_5,days_30,...,volume_5,std_vol_5,volume_365,std_vol_365,vol_ratio_5_365,std_vol_ratio_5_365,min_365,max_365,ratio_to_min_365,ratio_to_max_365
15859,1952-12-05,25.620001,25.620001,25.620001,25.620001,1.510000e+06,25.620001,25.680000,0.049497,24.835000,...,1.810000e+06,6.613622e+04,1.356904e+06,2.223773e+05,1.333919,0.297405,20.959999,25.740000,1.222328,0.995338
15858,1952-12-08,25.760000,25.760000,25.760000,25.760000,1.790000e+06,25.760000,25.672000,0.056302,24.886667,...,1.680000e+06,8.884819e+04,1.358027e+06,2.226700e+05,1.237088,0.399013,20.959999,25.740000,1.229008,1.000777
15857,1952-12-09,25.930000,25.930000,25.930000,25.930000,2.120000e+06,25.930000,25.688000,0.069065,24.952000,...,1.618000e+06,1.367962e+05,1.359863e+06,2.229455e+05,1.189826,0.613586,20.959999,25.760000,1.237118,1.006599
15856,1952-12-10,25.980000,25.980000,25.980000,25.980000,1.880000e+06,25.980000,25.726000,0.130115,25.020667,...,1.720000e+06,1.358632e+05,1.362658e+06,2.232067e+05,1.262239,0.608688,20.959999,25.930000,1.239504,1.001928
15855,1952-12-11,25.959999,25.959999,25.959999,25.959999,1.790000e+06,25.959999,25.780000,0.171318,25.085667,...,1.774000e+06,1.041288e+05,1.364137e+06,2.234562e+05,1.300456,0.465992,20.959999,25.980000,1.238550,0.999230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,3.916109,2073.984998,...,3.207544e+09,3.241733e+08,3.527800e+09,6.384811e+07,0.909219,5.077257,1862.489990,2130.820068,1.128935,0.986770
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,7.956808,2076.283993,...,3.232372e+09,3.390314e+08,3.526090e+09,6.446658e+07,0.916702,5.259026,1862.489990,2130.820068,1.116521,0.975920
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,9.333599,2077.908659,...,3.245514e+09,2.803620e+08,3.529468e+09,6.506381e+07,0.919548,4.309031,1862.489990,2130.820068,1.100473,0.961893
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,19.599946,2078.931331,...,3.536224e+09,1.696382e+08,3.532802e+09,6.567572e+07,1.000969,2.582967,1862.489990,2130.820068,1.123061,0.981636


In [80]:
numeric_df = data.iloc[:, 7:]

In [81]:
numeric_df.head()

,days_5,std_5,days_30,std_30,days_365,std_365,ratio_5_365,std_ratio_5_365,volume_5,std_vol_5,volume_365,std_vol_365,vol_ratio_5_365,std_vol_ratio_5_365,min_365,max_365,ratio_to_min_365,ratio_to_max_365
15859,25.680,0.049497,24.835000,0.612585,23.820712,0.982384,1.078053,0.050385,1810000.0,66136.223055,1.356904e+06,222377.332044,1.333919,0.297405,20.959999,25.74,1.222328,0.995338
15858,25.672,0.056302,24.886667,0.611202,23.830575,0.982327,1.077272,0.057315,1680000.0,88848.185125,1.358027e+06,222670.039612,1.237088,0.399013,20.959999,25.74,1.229008,1.000777
15857,25.688,0.069065,24.952000,0.595595,23.841123,0.982318,1.077466,0.070308,1618000.0,136796.198778,1.359863e+06,222945.520125,1.189826,0.613586,20.959999,25.76,1.237118,1.006599
15856,25.726,0.130115,25.020667,0.585208,23.852493,0.982413,1.078546,0.132444,1720000.0,135863.166458,1.362658e+06,223206.697838,1.262239,0.608688,20.959999,25.93,1.239504,1.001928
15855,25.780,0.171318,25.085667,0.579647,23.864630,0.981253,1.080260,0.174591,1774000.0,104128.766438,1.364137e+06,223456.217883,1.300456,0.465992,20.959999,25.98,1.238550,0.999230


In [82]:
numeric_cols = list(numeric_df.columns)

In [83]:
 from sklearn.preprocessing import MinMaxScaler

In [84]:
scaler = MinMaxScaler()

In [85]:
scaler.fit(numeric_df)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [86]:
numeric_df[numeric_cols] = scaler.fit_transform(numeric_df[numeric_cols])

In [87]:
numeric_df

,days_5,std_5,days_30,std_30,days_365,std_365,ratio_5_365,std_ratio_5_365,volume_5,std_vol_5,volume_365,std_vol_365,vol_ratio_5_365,std_vol_ratio_5_365,min_365,max_365,ratio_to_min_365,ratio_to_max_365
15859,0.001307,0.000380,0.000714,0.003710,0.000000,0.000652,0.695070,0.039905,0.000105,0.000046,0.000004,0.000262,0.399571,0.010551,0.0,0.000000,0.340956,0.933053
15858,0.001304,0.000455,0.000738,0.003699,0.000005,0.000652,0.694010,0.045970,0.000091,0.000062,0.000004,0.000263,0.361795,0.014466,0.0,0.000000,0.348585,0.942066
15857,0.001311,0.000596,0.000770,0.003578,0.000010,0.000652,0.694274,0.057340,0.000084,0.000098,0.000004,0.000263,0.343356,0.022735,0.0,0.000010,0.357850,0.951713
15856,0.001329,0.001271,0.000803,0.003497,0.000016,0.000652,0.695737,0.111715,0.000095,0.000097,0.000005,0.000264,0.371607,0.022547,0.0,0.000090,0.360575,0.943973
15855,0.001355,0.001727,0.000834,0.003453,0.000022,0.000648,0.698061,0.148597,0.000101,0.000074,0.000005,0.000264,0.386516,0.017048,0.0,0.000114,0.359485,0.939502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0.980557,0.043142,0.982287,0.191026,0.999335,0.253123,0.623558,0.049052,0.343796,0.238303,0.613725,0.084290,0.233884,0.194750,1.0,1.000000,0.234277,0.918856
3,0.982081,0.087830,0.983388,0.185699,0.999525,0.253052,0.625433,0.104014,0.346457,0.249226,0.613427,0.085106,0.236803,0.201754,1.0,1.000000,0.220098,0.900877
2,0.981166,0.103057,0.984166,0.173292,0.999685,0.252754,0.623932,0.122884,0.347866,0.206097,0.614015,0.085895,0.237913,0.165145,1.0,1.000000,0.201767,0.877634
1,0.977437,0.216597,0.984656,0.156196,0.999820,0.252133,0.618520,0.263302,0.379034,0.124702,0.614595,0.086703,0.269678,0.098628,1.0,1.000000,0.227568,0.910349


In [88]:
data["Date"].min()

Timestamp('1952-12-05 00:00:00')

In [89]:
data_clean = pd.concat([numeric_df, data[["Date", "Close"]]], axis = 1)

In [90]:
data_clean = data_clean.set_index("Date")

In [91]:
data_clean.shape

(15860, 19)

In [92]:
date_divider = datetime(year=2013, month=1, day=1)

In [93]:
train = data_clean[:date_divider].copy()

In [94]:
test = data_clean[date_divider: ].copy()

In [95]:
print(train.shape)
print(test.shape)

(15121, 19)
(739, 19)


In [96]:
train.head()

,days_5,std_5,days_30,std_30,days_365,std_365,ratio_5_365,std_ratio_5_365,volume_5,std_vol_5,volume_365,std_vol_365,vol_ratio_5_365,std_vol_ratio_5_365,min_365,max_365,ratio_to_min_365,ratio_to_max_365,Close
Date,,,,,,,,,,,,,,,,,,,
1952-12-05,0.001307,0.000380,0.000714,0.003710,0.000000,0.000652,0.695070,0.039905,0.000105,0.000046,0.000004,0.000262,0.399571,0.010551,0.0,0.000000,0.340956,0.933053,25.620001
1952-12-08,0.001304,0.000455,0.000738,0.003699,0.000005,0.000652,0.694010,0.045970,0.000091,0.000062,0.000004,0.000263,0.361795,0.014466,0.0,0.000000,0.348585,0.942066,25.760000
1952-12-09,0.001311,0.000596,0.000770,0.003578,0.000010,0.000652,0.694274,0.057340,0.000084,0.000098,0.000004,0.000263,0.343356,0.022735,0.0,0.000010,0.357850,0.951713,25.930000
1952-12-10,0.001329,0.001271,0.000803,0.003497,0.000016,0.000652,0.695737,0.111715,0.000095,0.000097,0.000005,0.000264,0.371607,0.022547,0.0,0.000090,0.360575,0.943973,25.980000
1952-12-11,0.001355,0.001727,0.000834,0.003453,0.000022,0.000648,0.698061,0.148597,0.000101,0.000074,0.000005,0.000264,0.386516,0.017048,0.0,0.000114,0.359485,0.939502,25.959999


### Making predictions

In [97]:
def train_and_test(df_train, df_test):
    y_train = train["Close"].copy()
    y_test = test["Close"].copy()
    X_train = df_train.drop(["Close"], axis=1)
    X_test = df_test.drop(["Close"], axis=1)
    lr= LinearRegression()
    lr.fit(X_train, y_train)
    prediction = lr.predict(X_test)
    mse = mean_squared_error(y_test, prediction)
    rmse = np.sqrt(mse)
    print("mse: {}".format(mse))
    print("rmse: {}".format(rmse))
    return mse, rmse

In [98]:
train_and_test(train, test)

mse: 379.3860905838879
rmse: 19.477835880402317


(379.3860905838879, 19.477835880402317)